In [1]:
from simsym import g, t, Object, eqs, subs
from sympy.physics.units import Quantity, length, time
from sympy.physics.units.systems import SI
from sympy import Eq, latex, Symbol
from IPython.display import Math, HTML, clear_output
from ipywidgets import *
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
from difflib import SequenceMatcher

In [2]:
%%html
<style>
.flexgrid {
    display: grid;
    grid-template-columns: repeat(auto-fill, minmax(100px, 1fr));
}
div.widget-label-small > .widget-label {
    width: 1px;
}
div.widget-label-example > .widget-label {
    width: 60px;
}
</style>

In [22]:
class GUI():
    objects: dict[str, Object] = dict()
    variables = {}
    equations = {}
    gui: Widget
    define_pane: Widget
    obj_name_input: Widget
    obj_name_input_text: Text
    obj_name_input_Button: Button
    show_obj: HBox
    show_obj_children: dict[str, HBox] = dict()
    global_val: HBox
    global_val_children: VBox
    show_equation: HBox
    show_equation_children: VBox
    observe_pane: Widget
    subs_vars = dict()
    
    def __init__(self, fps=10):
        self.fps = fps
        self.max_time = 4
        self.init_gui()
        
    def update_show_obj(self, show_obj):
        show_obj = FloatInput()
        
    def display(self):
        display(self.gui)
    
    def init_gui(self):
        self.init_observe_pane()
        self.init_define_pane()
        self.gui = HBox([self.define_pane, self.observe_pane], layout=Layout(width='100%'))
    
    def init_define_pane(self):
        self.init_obj_name_input()
        self.init_show_obj()
        self.init_global_val()
        self.init_show_equation()
        tab = Tab(children=[VBox([self.obj_name_input, self.show_obj, self.global_val, self.show_equation])], layout=Layout(width='45%'))
        tab.set_title(0, '物理系定義ペイン')
        self.define_pane = tab

    def init_obj_name_input(self):
        text = Text(placeholder='名前を入力', layout=Layout(width='90px'))
        self.obj_name_input_text = text
        button = Button(description='物体追加', layout=Layout(width='80px'))
        self.obj_name_input_button = button
        def on_click(b):
            self.objects[text.value] = Object(text.value)
            self.select_target_object.options += (text.value,)
            text.value = ''
            self.update_show_obj()
        button.on_click(on_click)
        self.obj_name_input = HBox([text, button])
    
    def init_show_obj(self):
        self.show_obj = HBox([], layout=Layout(width='100%'))

    def update_show_obj(self):
        contents = []
        for key, value in self.objects.items():
            if key not in self.show_obj_children.keys():
                x = HTMLMath(value=r'$x_{' + key + r'}~\mathrm{[L]}$')
                self.variables[r'x_{' + key + r'}'] = Symbol(r'x_{' + key + r'}')
                y = HTMLMath(value=r'$y_{' + key + r'}~\mathrm{[L]}$')
                self.variables[r'y_{' + key + r'}'] = Symbol(r'y_{' + key + r'}')
                vx = HTMLMath(value=r'$v_{' + key + r'x}~\mathrm{[L/T]}$')
                self.variables[r'v_{' + key + r'x}'] = Symbol(r'v_{' + key + r'x}')
                vy = HTMLMath(value=r'$v_{' + key + r'y}~\mathrm{[L/T]}$')
                self.variables[r'v_{' + key + r'y}'] = Symbol(r'v_{' + key + r'y}')
                content = VBox([x, y, vx, vy])
                self.show_obj_children[key] = content
                content.add_class('flexgrid')
            else:
                content = self.show_obj_children[key]
            
            text = Text(placeholder='変数名', layout=Layout(width='100px'))
            unit = Text(placeholder='単位', layout=Layout(width='100px'))
            button = Button(description='変数追加', layout=Layout(width='80px'))
            def on_click(b):
                self.show_obj_children[key].children += (HTMLMath(value=r'$' + text.value + r'~\mathrm{[' + unit.value + r']}$'), )
                text.value = ''
                unit.value = ''
            button.on_click(on_click)
            content_outer = VBox([content, HBox([text, unit, button])])
            
            contents.append(content_outer)
        tab = Tab(children=contents, layout=Layout(width='95%'))
        for i in range(len(self.objects)):
            tab.set_title(i, list(self.objects.keys())[i])
        self.show_obj.children = [tab]
    
    def init_global_val(self):
        t = HTMLMath(value=r'$t~\mathrm{[T]}$')
        self.variables['t'] = Symbol('t')
        g = HTMLMath(value=r'$g~\mathrm{[L/T^2]}$')
        self.variables['g'] = Symbol('g')
        self.global_val_children = VBox([t, g])
        self.global_val_children.add_class('flexgrid')
        
        text = Text(placeholder='変数名', layout=Layout(width='100px'))
        unit = Text(placeholder='単位', layout=Layout(width='100px'))
        button = Button(description='変数追加', layout=Layout(width='80px'))
        def on_click(b):
            self.global_val_children.children += (HTMLMath(value=r'$' + text.value + r'~\mathrm{[' + unit.value + r']}$'), )
            text.value = ''
            unit.value = ''
        button.on_click(on_click)
        content = VBox([self.global_val_children, HBox([text, unit, button])])
        
        tab = Tab(children=[content], layout=Layout(width='95%'))
        tab.set_title(0, 'グローバル')
        self.global_val = tab
        
    def init_show_equation(self):
        self.show_equation_children = VBox([])
        lhs = Text(placeholder='左辺', layout=Layout(width='200px'))
        rhs = Text(placeholder='右辺', layout=Layout(width='200px'))
            
        button = Button(description='追加', layout=Layout(width='80px'))
        def on_click(b):
            self.show_equation_children.children += (HTMLMath(value=r'$' + lhs.value + '=' + rhs.value + '$'), )
            self.equations[lhs.value + '=' + rhs.value] = lhs.value + '=' + rhs.value
            if 'v_{0x}' in rhs.value:
                self.update_subs('v_{0x}')
            if 'v_{0y}' in rhs.value:
                self.update_subs('v_{0y}')
            if 'g' in rhs.value:
                self.update_subs('g')
            lhs.value = ''
            rhs.value = ''
        button.on_click(on_click)
        content = VBox([self.show_equation_children, HBox([lhs, rhs, button])])
        
        tab = Tab(children=[content], layout=Layout(width='95%'))
        tab.set_title(0, '方程式')
        self.show_equation = tab
        
    def init_observe_pane(self):
        self.init_select_target()
        self.init_time_slider()
        self.init_graph()
        self.init_subs()
        
        tab = Tab(children=[VBox([
            self.select_target,
            self.graph,
            HBox([self.time_slider, self.time_output, self.time_play]),
            self.subs
        ])], layout=Layout(width='45%'))
        tab.set_title(0, '観測ペイン')
        self.observe_pane = tab
    
    def init_select_target(self):
        self.select_target_object = Dropdown(options=[], description='観測対象', layout=Layout(width='160px'))
        self.select_target_object.add_class('widget-label-example')
        self.select_target_pos = Checkbox(value=True, description='位置', layout=Layout(width='70px'))
        self.select_target_pos.add_class('widget-label-small')
        self.select_target_vel = Checkbox(value=True, description='速度', layout=Layout(width='70px'))
        self.select_target_vel.add_class('widget-label-small')
        self.select_example = Dropdown(options=['斜方投射', '自由落下'], description='動作例', layout=Layout(width='200px'))
        self.select_example.add_class('widget-label-example')
        self.select_target = HBox([
            self.select_target_object,
            self.select_target_pos,
            self.select_target_vel,
            self.select_example
        ], layout=Layout(width='95%'))
    
    def init_time_slider(self):
        self.time_play = Play(value=0, min=0, max=self.max_time*self.fps, interval=1000 / self.fps, description="再生:")
        self.time_output = Output(layout=Layout(width='100px'))
        self.time_slider = IntSlider(value=0, min=0, max=self.max_time*self.fps, description='$t$', readout=False)
        tmp_output = Output()
        widgets.jslink((self.time_play, 'value'), (self.time_slider, 'value'))
        with tmp_output:
            @interact(slider=self.time_slider)
            def update(slider):
                with self.time_output:
                    clear_output()
                    print(f'{slider/self.fps:.2f}')
            clear_output()
    
    def update_graph(self, slider, obj_name, example_name):
        slider = slider / self.fps
        # modify here
        t = np.linspace(0, slider, num=1000)
        x = 3 * t
        y = 4 * t + t**2
        vx = 3
        vy = 4 + 2 + slider
        try:
            with self.graph:
                fig, ax = plt.subplots(constrained_layout=True, figsize=(6, 4))
                ax.grid(True)
                
                if self.select_example.value == '斜方投射':
                    t_all = np.linspace(0, 4, num=1000)
                    x_all = 3 * t_all
                    y_all = 4 * t_all - t_all**2
                    ax.plot(x_all, y_all, '#797979', linestyle='dashed')
                
                if self.select_target_object.value == 'A':
                    ax.plot(x, y, color='red')
                    ax.plot(x[-1], y[-1], marker="o", markersize=10, color='red')
                    if self.drawx:
                        if self.drawy:
                            coordinates = np.array([[vx, 0], [0, vy], [vx, vy]])
                        else:
                            coordinates = np.array([[vx, 0], [0, 0], [vx, 0]])
                    elif self.drawy:
                        coordinates = np.array([[0, 0], [0, vy], [0, vy]])
                    else:
                        coordinates = np.array([[0, 0], [0, 0], [0, 0]])
                    o = np.array([[x[-1]]*3, [y[-1]]*3])
                    ax.quiver(*o, coordinates[:, 0], coordinates[:, 1], angles='xy', scale_units='xy', color=['blue']*3, scale=4)
                plt.show(fig)
        except AttributeError:
            pass
    
    def init_graph(self):
        self.graph = interactive_output(self.update_graph, {
            'slider': self.time_slider,
            'obj_name': self.select_target_object,
            'example_name': self.select_example})
        self.update_graph(0, "", "")
    
    def init_subs(self):
        self.subs = HBox([])
        self.subs.add_class('flexgrid')
    
    drawx = False
    drawy = False
    def update_subs(self, var):
        if var in self.subs_vars.keys():
            return
        text = FloatText(value=0, description=f'${var}=$', layout=Layout(width='140px'))
        self.subs_vars[var] = text
        self.subs.children += (text,)
        @interact(subs_val=text)
        def foo(subs_val):
            if subs_val == 3:
                self.drawx = True                
            if subs_val == 4:
                self.drawy = True
            with self.graph:
                clear_output()
                self.update_graph(self.time_slider.value, self.select_target_object.value, self.select_example.value)


gui = GUI(fps=20)
gui.display()

In [13]:
gui.equations

{'v_{Ax}=v_{0x}': 'v_{Ax}=v_{0x}'}